In [1]:
## import packages
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from imutils import paths
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import save_model, load_model, Model
from tensorflow.keras.applications import ResNet50V2, VGG16
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization, AveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam

In [2]:
## data preprocessing
imagePath = "./all/train"
imagePaths = list(paths.list_images(imagePath))
data = []
labels = []

for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))
    data.append(image)
    labels.append(label)

data = np.array(data) / 255.0
labels = np.array(labels)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
labels = to_categorical(integer_encoded)

(x_train, x_val, y_train, y_val) = train_test_split(data, labels, test_size=0.20, stratify=labels)

In [3]:
## AI model training
adam_s = Adam(learning_rate = 0.00001)

baseModel = InceptionV3(input_shape=(256, 256, 3),include_top=False, weights='imagenet')

for layer in baseModel.layers:
    layer.trainable = False

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4,4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(128, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation='softmax')(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

model.compile(loss='categorical_crossentropy', optimizer=adam_s, metrics=['accuracy'])
model.summary()

87916544/87910968 [==============================] - 2426s 28us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
__________________

In [4]:
# training AI model
H = model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), batch_size=8)

Epoch 1/50
20/20 [==============================] - 8s 422ms/step - loss: 1.3439 - accuracy: 0.4187 - val_loss: 1.0653 - val_accuracy: 0.2750
Epoch 2/50
20/20 [==============================] - 8s 394ms/step - loss: 1.4640 - accuracy: 0.3250 - val_loss: 1.0418 - val_accuracy: 0.3750
Epoch 3/50
20/20 [==============================] - 9s 436ms/step - loss: 1.3095 - accuracy: 0.4375 - val_loss: 1.0170 - val_accuracy: 0.4250
Epoch 4/50
20/20 [==============================] - 9s 472ms/step - loss: 1.2879 - accuracy: 0.3938 - val_loss: 1.0009 - val_accuracy: 0.4250
Epoch 5/50
20/20 [==============================] - 9s 429ms/step - loss: 1.4071 - accuracy: 0.3313 - val_loss: 0.9815 - val_accuracy: 0.4500
Epoch 6/50
20/20 [==============================] - 8s 417ms/step - loss: 1.3089 - accuracy: 0.4187 - val_loss: 0.9717 - val_accuracy: 0.5000
Epoch 7/50
20/20 [==============================] - 8s 412ms/step - loss: 1.3749 - accuracy: 0.3375 - val_loss: 0.9571 - val_accuracy: 0.4750
Epoch 

In [ ]:
model.summary()

In [7]:
## save AI model
save_model(model, "head-base-covid/1")

model.save("inceptionv3_base.h5")

INFO:tensorflow:Assets written to: head-base-covid/1/assets
